In [ ]:
def asset_location(
    allocation_dict,
    tickers_by_tax_efficiency,
    accounts_by_tax_advantage,
    roth_ira_limit,
    total_money,
    account_limits=None,
    real_allocation=False,
    csv_filepath='portfolio_state.csv'
):
    """
    Optimize asset location by placing least tax-efficient assets in most tax-advantaged accounts.
    
    Parameters:
    -----------
    allocation_dict : dict
        Dictionary of {ticker: percentage} where percentage is decimal (e.g., 0.30 for 30%)
    tickers_by_tax_efficiency : list
        List of tickers ordered from most to least tax efficient
    accounts_by_tax_advantage : list
        List of account names ordered from most to least tax advantaged
    roth_ira_limit : float
        Dollar limit for Roth IRA account
    total_money : float
        Total amount of money to allocate
    account_limits : dict, optional
        Dictionary of {account_name: limit}. If not provided, uses roth_ira_limit for 'Roth IRA'
        and no limits for other accounts
    real_allocation : bool, default False
        If True, reads/writes CSV to preserve state. If False, only hypothetical (experimenting)
    csv_filepath : str, default 'portfolio_state.csv'
        Path to CSV file for storing portfolio state
    
    Returns:
    --------
    dict
        Nested dictionary: {account: {ticker: amount}}
    """
    import pandas as pd
    import os
    import pandas as pd
    import os
    
    # Load current holdings if real allocation
    current_holdings = None
    if real_allocation and os.path.exists(csv_filepath):
        current_holdings = pd.read_csv(csv_filepath)
        print(f"\n{'='*70}")
        print(f"Loaded existing portfolio state from {csv_filepath}")
        print(f"{'='*70}")
    
    # Calculate dollar amounts for each ticker
    ticker_amounts = {ticker: total_money * pct for ticker, pct in allocation_dict.items()}
    
    # Set up account limits
    if account_limits is None:
        account_limits = {}
        for account in accounts_by_tax_advantage:
            if 'roth' in account.lower():
                account_limits[account] = roth_ira_limit
            else:
                account_limits[account] = float('inf')  # No limit
    
    # Initialize result and remaining capacity
    result = {account: {} for account in accounts_by_tax_advantage}
    remaining_capacity = account_limits.copy()
    
    # Reverse the tax efficiency list to go from least to most tax efficient
    tickers_least_to_most_efficient = list(reversed(tickers_by_tax_efficiency))
    
    # Allocate each ticker starting with least tax efficient
    for ticker in tickers_least_to_most_efficient:
        if ticker not in ticker_amounts:
            continue
            
        amount_to_place = ticker_amounts[ticker]
        
        # Try to place in accounts from most to least tax advantaged
        for account in accounts_by_tax_advantage:
            if amount_to_place <= 0:
                break
                
            # Calculate how much we can place in this account
            available_space = remaining_capacity[account]
            amount_placed = min(amount_to_place, available_space)
            
            if amount_placed > 0:
                result[account][ticker] = amount_placed
                remaining_capacity[account] -= amount_placed
                amount_to_place -= amount_placed
    
    # Save allocation to CSV if real allocation
    if real_allocation:
        # Convert result to DataFrame format for CSV
        rows = []
        for account, allocations in result.items():
            if allocations:
                for ticker, amount in allocations.items():
                    rows.append({
                        'Account': account,
                        'Ticker': ticker,
                        'Amount': amount,
                        'Percentage': (amount / total_money * 100)
                    })
        
        allocation_df = pd.DataFrame(rows)
        allocation_df.to_csv(csv_filepath, index=False)
        print(f"\n{'='*70}")
        print(f"Portfolio state saved to {csv_filepath}")
        print(f"{'='*70}\n")
    else:
        print(f"\n[EXPERIMENTING MODE] CSV not updated (real_allocation=False)\n")
    
    return result


def print_asset_location(result, total_money, csv_filepath='portfolio_state.csv'):
    """
    Pretty print the asset location results using pandas DataFrame.
    
    Parameters:
    -----------
    result : dict
        Output from asset_location function
    total_money : float
        Total amount allocated
    csv_filepath : str
        Path to portfolio state CSV
    """
    import pandas as pd
    from IPython.display import display, HTML
    import os
    
    # Build data for the table
    rows = []
    for account, allocations in result.items():
        if allocations:
            for ticker, amount in allocations.items():
                rows.append({
                    'Account': account,
                    'Ticker': ticker,
                    'Amount': amount,
                    'Percentage': amount / total_money * 100
                })
    
    # Create DataFrame
    df = pd.DataFrame(rows)
    
    # Format the display
    df_display = df.copy()
    df_display['Amount'] = df_display['Amount'].apply(lambda x: f"${x:,.2f}")
    df_display['Percentage'] = df_display['Percentage'].apply(lambda x: f"{x:.2f}%")
    
    # Calculate account totals
    account_totals = df.groupby('Account')['Amount'].sum().reset_index()
    account_totals['Percentage'] = account_totals['Amount'] / total_money * 100
    account_totals['Ticker'] = 'TOTAL'
    account_totals['Amount'] = account_totals['Amount'].apply(lambda x: f"${x:,.2f}")
    account_totals['Percentage'] = account_totals['Percentage'].apply(lambda x: f"{x:.2f}%")
    account_totals = account_totals[['Account', 'Ticker', 'Amount', 'Percentage']]
    
    # Display title
    print(f"\n{'='*70}")
    print(f"Asset Location Plan - New Allocation: ${total_money:,.2f}")
    print(f"{'='*70}\n")
    
    # Display main allocation table
    display(df_display)
    
    # Display account summary
    print(f"\n{'-'*70}")
    print("Account Summary:")
    print(f"{'-'*70}\n")
    display(account_totals)
    
    # Show saved state if CSV exists
    if os.path.exists(csv_filepath):
        print(f"\n{'-'*70}")
        print(f"Current Portfolio State (from {csv_filepath}):")
        print(f"{'-'*70}\n")
        saved_df = pd.read_csv(csv_filepath)
        saved_display = saved_df.copy()
        saved_display['Amount'] = saved_display['Amount'].apply(lambda x: f"${x:,.2f}")
        saved_display['Percentage'] = saved_display['Percentage'].apply(lambda x: f"{x:.2f}%")
        display(saved_display)


In [ ]:
# Example usage
allocation_dict = {'SPMO': 0.1, 'VTI': 0.4, 'VONG': 0.1, 'VXUS': 0.05, 'AVUV': 0.1, 'AVDV': 0.1, 'JQUA': 0.1, 'DYNF': 0.15}

# Order tickers by tax efficiency (most efficient first)
tickers_by_tax_efficiency = [
    'VXUS',
    'VTI',
    'VONG',
    'AVUV',
    'AVDV',
    'JQUA',
    'DYNF',
    'SPMO'
]

# Order accounts by tax advantage (most advantaged first)
accounts_by_tax_advantage = [
    'Roth IRA',
    'Taxable Brokerage'
]

roth_ira_limit = 7000
total_money = 5500+18000

# EXPERIMENTING: Test allocation without saving to CSV
print("\n" + "="*70)
print("EXPERIMENTING MODE - No CSV updates")
print("="*70)

result = asset_location(
    allocation_dict,
    tickers_by_tax_efficiency,
    accounts_by_tax_advantage,
    roth_ira_limit,
    total_money,
    account_limits={
        'Roth IRA': 7000,
        'Taxable Brokerage': float('inf')
    },
    real_allocation=False  # Experimenting
)

# Print results
print_asset_location(result, total_money)


Asset Location Plan - Total Portfolio: $23,500.00



,Account,Ticker,Amount,Percentage
0,Roth IRA,SPMO,"$2,350.00",10.00%
1,Roth IRA,DYNF,"$3,525.00",15.00%
2,Roth IRA,JQUA,"$1,125.00",4.79%
3,Taxable Brokerage,JQUA,"$1,225.00",5.21%
4,Taxable Brokerage,AVDV,"$2,350.00",10.00%
5,Taxable Brokerage,AVUV,"$2,350.00",10.00%
6,Taxable Brokerage,VONG,"$2,350.00",10.00%
7,Taxable Brokerage,VTI,"$9,400.00",40.00%
8,Taxable Brokerage,VXUS,"$1,175.00",5.00%



----------------------------------------------------------------------
Account Summary:
----------------------------------------------------------------------



,Account,Ticker,Amount,Percentage
0,Roth IRA,TOTAL,"$7,000.00",29.79%
1,Taxable Brokerage,TOTAL,"$18,850.00",80.21%


In [ ]:
# REAL ALLOCATION: Save state to CSV
print("\n" + "="*70)
print("REAL ALLOCATION MODE - CSV will be updated")
print("="*70)

# Uncomment below to execute a REAL allocation (this will create/update CSV)
# result_real = asset_location(
#     allocation_dict,
#     tickers_by_tax_efficiency,
#     accounts_by_tax_advantage,
#     roth_ira_limit,
#     total_money,
#     account_limits={
#         'Roth IRA': 7000,
#         'Taxable Brokerage': float('inf')
#     },
#     real_allocation=True,  # REAL - will save to CSV
#     csv_filepath='portfolio_state.csv'
# )
# print_asset_location(result_real, total_money, csv_filepath='portfolio_state.csv')